# Additional Tests for the Simulation

In [1]:
from simulation import Airport, Queue, run_simulation, Passenger
import numpy as np
import random

## Test 1
If the passengers are being allocated to the additional service 3% of the time

In [2]:
queue = Queue(timestamp=0, service_rate=1/2)
additional_service_count = 0
run_until = 10000 # number of samples

for _ in range(run_until):
    if queue.needs_additional_screening():
        additional_service_count += 1

proportion = additional_service_count / run_until
print(f"Proportion needing additional screening: {proportion:.2f}")

# Calculate confidence interval
z_score = 1.96  # for 95% confidence
margin_of_error = z_score * np.sqrt(proportion * (1 - proportion) / run_until)
confidence_interval = (proportion - margin_of_error, proportion + margin_of_error)
print(f"95% confidence interval: {confidence_interval}")

Proportion needing additional screening: 0.03
95% confidence interval: (0.027034968247638666, 0.033765031752361334)


In [3]:
# Alternative way to test this
queue_multi, arrivals_multi, completed_passengers = run_simulation(arrival_rate=10, service_rate=1/2, run_until=1000, num_queues=3)

# Count passengers who needed additional screening
additional_screening_count = sum(1 for passenger in completed_passengers if passenger.needs_additional_screening)

total_passengers = len(completed_passengers)
proportion = additional_screening_count / total_passengers if total_passengers > 0 else 0

print(f"Passengers needing additional screening: {additional_screening_count}/{total_passengers} ({proportion:.1%})")

Passengers needing additional screening: 31/1000 (3.1%)


## Test 2
Check whether passengers are being assigned to the shortest queue

In [4]:
airport = Airport(arrival_rate=10, service_rate=1/2, num_queues=3)

# creating passenger objects
passenger1 = Passenger(1, arrival_time=0)
passenger2 = Passenger(2, arrival_time=1)
passenger3 = Passenger(3, arrival_time=2)
passenger4 = Passenger(4, arrival_time=3)
passenger5 = Passenger(5, arrival_time=4)
passenger6 = Passenger(6, arrival_time=5)

# Add passengers to queues manually to test shortest queue selection
airport.passenger_queues[0].waiting = [passenger1, passenger2]  # 2 passengers
airport.passenger_queues[1].waiting = [passenger3]  # 1 passenger
airport.passenger_queues[2].waiting = [passenger4, passenger5, passenger6]  # 3 passengers

# Find shortest queue
shortest_queue = airport.find_shortest_queue()
shortest_index = airport.passenger_queues.index(shortest_queue)
print(f"Shortest queue index: {shortest_index}, with {len(shortest_queue.waiting)} passengers waiting")

# Verify it's actually the shortest
queue_lengths = [q.total_passengers() for q in airport.passenger_queues]
print(f"All queue lengths: {queue_lengths}")
print(f"Test passed: {queue_lengths[shortest_index] == min(queue_lengths)}")

Shortest queue index: 1, with 1 passengers waiting
All queue lengths: [2, 1, 3]
Test passed: True


## Test 3
Check if passengers are randomly assigned to a queue when all queues are the same length

In [5]:
airport = Airport(arrival_rate=10, service_rate=1/2, num_queues=3)

# creating passenger objects
passenger1 = Passenger(1, arrival_time=0)
passenger2 = Passenger(2, arrival_time=1)
passenger3 = Passenger(3, arrival_time=2)
passenger4 = Passenger(4, arrival_time=3)
passenger5 = Passenger(5, arrival_time=4)
passenger6 = Passenger(6, arrival_time=5)

# Add passengers to queues manually to test shortest queue selection
airport.passenger_queues[0].waiting = [passenger1, passenger2]  # 2 passengers
airport.passenger_queues[1].waiting = [passenger3, passenger4]  # 2 passengers
airport.passenger_queues[2].waiting = [passenger5, passenger6]  # 2 passengers

# Here, we want to test if the proportion of passengers assigned to each queue is roughly equal over many trials
num_trials = 1000
selected_indices = []

for _ in range(num_trials):
    selected_queue = airport.find_shortest_queue()
    selected_index = airport.passenger_queues.index(selected_queue)
    selected_indices.append(selected_index)

# Count how many times each queue was selected
queue_0_count = selected_indices.count(0)
queue_1_count = selected_indices.count(1)
queue_2_count = selected_indices.count(2)

perc_1 = (queue_0_count/num_trials)*100
perc_2 = (queue_1_count/num_trials)*100
perc_3 = (queue_2_count/num_trials)*100

print(f"Queue selection distribution over {num_trials} trials:")
print(f"Queue 0: {queue_0_count} times ({perc_1:.1f}%)")
print(f"Queue 1: {queue_1_count} times ({perc_2:.1f}%)")
print(f"Queue 2: {queue_2_count} times ({perc_3:.1f}%)")

Queue selection distribution over 1000 trials:
Queue 0: 314 times (31.4%)
Queue 1: 331 times (33.1%)
Queue 2: 355 times (35.5%)


## Test 4
Testing queue placing dynamics, meant to check whether:
* There are unique arrival and departure events ofr each new passenger
* When a passenger arrives, they are placed on the shortest queue or assigned at random when all queues have the same length
* When a passenger leaves, according to the scheduled time, they leave the queue completely

In [2]:
# Parameters
run_until = 100
num_queues = 3
avg_waiting_times = []
arrival_rate = 10 # minutes
service_rate = 1/2 # minutes

airport = Airport(arrival_rate, service_rate, num_queues)

# Schedule the specified number of arrivals
current_time = 0
for _ in range(run_until):
    airport.add_arrival(current_time)
    current_time = airport.curr_arrival_time  # Use the scheduled arrival time for next scheduling
    
while airport.priority_q:
    airport.run_next_service()
    airport.print_schedule()

Airport at time (minutes) 0.017: 99 total waiting, 1 total being served
  Queue 1: 33 waiting, 0 being served and 0 needed additional screening so far.
  Queue 2: 33 waiting, 0 being served and 0 needed additional screening so far.
  Queue 3: 33 waiting, 1 being served and 0 needed additional screening so far.
	 Scheduled Timestamp (minutes) 0.04: arrival for passenger 2 at queue 0
	 Scheduled Timestamp (minutes) 0.28: arrival for passenger 3 at queue 1
	 Scheduled Timestamp (minutes) 0.29: arrival for passenger 4 at queue 0
	 Scheduled Timestamp (minutes) 0.36: arrival for passenger 5 at queue 1
	 Scheduled Timestamp (minutes) 0.39: arrival for passenger 6 at queue 2
	 Scheduled Timestamp (minutes) 0.44: arrival for passenger 7 at queue 0
	 Scheduled Timestamp (minutes) 0.57: arrival for passenger 8 at queue 1
	 Scheduled Timestamp (minutes) 0.86: arrival for passenger 9 at queue 2
	 Scheduled Timestamp (minutes) 0.89: arrival for passenger 10 at queue 1
	 Scheduled Timestamp (minutes